# Fasttext
Build a fasttext model and do some basic analyses.
Before running this code parse the ogsl data set to produce a dataframe of OGSL sign readings, sign names and utf8 representations, and parse the Ur 3 dataset to produce a text file in cuneiform utf8. 

In [1]:
import pickle
from fastText import train_unsupervised
from fastText import util
import pandas as pd
import numpy as np

In [2]:
model = train_unsupervised('output/sux.txt', model='skipgram', ws=10, neg=70, epoch=100)
# increase negative sampling to 100 or 200 (70?)
# increase epochs 1000?

In [3]:
labels = model.get_words(include_freq=True)

In [4]:
df = pd.DataFrame({"label": list(labels[0]), "count":list(labels[1])})

In [5]:
df

,label,count
0,𒁹,145853
1,xxx,115817
2,𒋡,95235
3,</s>,88104
4,𒈫,78252
5,X,75655
6,𒈬,73042
7,𒌋,70945
8,𒐊,65869
9,𒂆,59101


In [6]:
class FastTextNN:
    """by Ali Abul Hawa; see https://github.com/facebookresearch/fastText/pull/552"""    
    def __init__(self, ft_model, ft_matrix=None):
        self.ft_model = ft_model        
        self.ft_words = ft_model.get_words()
        self.word_frequencies = dict(zip(*ft_model.get_words(include_freq=True)))
        self.ft_matrix = ft_matrix
        if self.ft_matrix is None:
            self.ft_matrix = np.empty((len(self.ft_words), ft_model.get_dimension()))
            for i, word in enumerate(self.ft_words):
                self.ft_matrix[i,:] = ft_model.get_word_vector(word)
    
    def find_nearest_neighbor(self, query_word, vectors, n=10,  cossims=None):
        """
        vectors is a 2d numpy array corresponding to the vectors you want to consider

        cossims is a 1d numpy array of size len(vectors), which can be passed for efficiency
        returns the index of the closest n matches to query within vectors and the cosine similarity (cosine the angle between the vectors)

        """
        
        query  = self.ft_model.get_word_vector(query_word)
        if cossims is None:
            cossims = np.matmul(vectors, query, out=cossims)

        norms = np.sqrt((query**2).sum() * (vectors**2).sum(axis=1))
        cossims = cossims/norms
        if query_word in self.ft_words:
            result_i = np.argpartition(-cossims, range(n+1))[1:n+1]
        else:
            result_i = np.argpartition(-cossims, range(n+1))[0:n]
        return list(zip(result_i, cossims[result_i]))

    def nearest_words(self, word, n=10, word_freq=None):
        result = self.find_nearest_neighbor(word, self.ft_matrix, n=n)
        if word_freq:
            return [(self.ft_words[r[0]], r[1]) for r in result if self.word_frequencies[self.ft_words[r[0]]] >= word_freq]
        else:
            return [(self.ft_words[r[0]], r[1]) for r in result]

# Basic usage of nearest_words

In [7]:
fasttext_nn = FastTextNN(model)
fasttext_nn.nearest_words('𒈗', word_freq=15)

[('𒋀𒀊𒆠𒈠', 0.7799620299935822),
 ('𒆗𒂵', 0.7683318194900972),
 ('𒀭𒋗𒀭𒂗𒍪', 0.758994450086335),
 ('𒌒𒁕', 0.7051389328588079),
 ('𒀭', 0.7021102926988015),
 ('𒆠', 0.694199029614253),
 ('𒈬', 0.6805457073776687),
 ('𒀭𒄿𒉈𒀭𒂗𒍪', 0.6756081610552702),
 ('𒋀𒀊𒆠𒈠𒆤', 0.673225210784096),
 ('𒇹𒁀', 0.672182360684783)]

# Allow input in transliteration

In [8]:
# create conversion dictionary value to utf8
import pickle
with open("output/ogsl.p", "rb") as p:
    o = pickle.load(p)
d = dict(zip(o["value"], o["utf8"]))

In [9]:
def nearestcun(text, n=10, word_freq=None):
    """input transliteration of a word form, signs separated by hyphens.
    Examples: "ma-an-gi₄", "d-en-lil₂", or "urim₅-ki".
    Transliteration style (gu₄ vs. gud; gen vs. ŋen) is unimportant
    The result is fed to nearest_words.
    n is the number of nearest neigbors that nearest_words will return
    word_freq is the minimum word frequency for a neighbor to be considered"""
    signs = text.lower().split('-')
    seq = [d[s] if s in d else s for s in signs]
    seq = ''.join(seq)
    return fasttext_nn.nearest_words(seq, n, word_freq)

In [13]:
nearestcun("an-za-am-da", n=15, word_freq=10)

[('𒀭𒈗𒄘𒃮𒀀𒆠', 0.6529808289716572),
 ('𒀊𒀊𒄰', 0.6426204320204592),
 ('𒂗𒀭𒀏𒆠𒉘', 0.5987514507795922),
 ('𒀭𒀏𒄉𒃲', 0.5972548810335401),
 ('𒀀𒌅', 0.578125342313602),
 ('𒆭𒊏', 0.5630318542360572),
 ('𒁕𒁷𒀹𒁯𒂠', 0.5601867481618125),
 ('𒀭𒊩𒌆𒄭𒂦𒂵', 0.5460045249931424),
 ('KWU636𒊏', 0.5431356519135444),
 ('𒀭𒊩𒌆𒆤', 0.534465436301629),
 ('𒁀𒉌', 0.5298289195769502),
 ('𒀭𒊩𒌆𒅔𒋛', 0.5282395804713714),
 ('𒂍𒄩𒉡𒅥', 0.5191612118055874)]

# TODO
Transliterate output by finding the word in the corpus

In [11]:
model.get_subwords("𒄀𒇡𒇡")

(['𒄀𒇡𒇡', '<𒄀𒇡', '<𒄀𒇡𒇡', '<𒄀𒇡𒇡>', '𒄀𒇡𒇡', '𒄀𒇡𒇡>', '𒇡𒇡>'],
 array([   9506,  277830, 1938156, 1303352,  515818, 1330142, 1013048]))

In [12]:
model.save_model("output/suxmodel")